## extract dataset of s1d1 & s2d1,as training data and testing data respectively

In [1]:
import scanpy as sc
import anndata as ad
import numpy as np
import xchrom as xc

In [2]:
path = '/picb/bigdata/project/miaoyuanyuan/train/XChrom_analysis/2_cross_samples/s1d1_s2d1/0_preprocess/'
bmmc = sc.read_h5ad(f'{path}/raw_data/bmmc_GSE194122.h5ad')

In [3]:
train = 'site1_donor1_multiome'
test = 'site2_donor1_multiome'

In [4]:
bmmc

AnnData object with n_obs × n_vars = 69249 × 129921
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [5]:
adata = bmmc[(bmmc.obs['Samplename'] == train) | (bmmc.obs['Samplename'] == test)]

In [6]:
adata.obs.groupby('Samplename').size()

Samplename
site1_donor1_multiome    6224
site2_donor1_multiome    4220
dtype: int64

In [7]:
adata.layers['counts'] = adata.layers['counts'].astype(np.int32)
adata.X = adata.layers['counts'].copy()
# Create a new AnnData and retain the layers of interest
adata = ad.AnnData(
    X=adata.X.copy(),
    obs=adata.obs.copy(),
    var=adata.var.copy()
)
adata.obs = adata.obs[['cell_type','batch','Samplename']] 

adata_rna = adata[:, adata.var['feature_types']=='GEX']
adata_atac = adata[:, adata.var['feature_types']=='ATAC']

In [8]:
## Break down the features in var into chr, start, and end
features = adata_atac.var.index
chromosome_start_end = features.str.split("-", n=2, expand=True)
chromosome_start_end_df = chromosome_start_end.to_frame(index=False)
chromosome_start_end_df.columns = ["chr", "start", "end"]
chromosome_start_end_df.index = adata_atac.var.index
adata_atac.var['chr'] = chromosome_start_end_df['chr']
adata_atac.var['start'] = chromosome_start_end_df['start'].astype(int)
adata_atac.var['end'] = chromosome_start_end_df['end'].astype(int)
print(adata_atac.var[['chr', 'start', 'end']].head())

/tmp/ipykernel_16760/2766228937.py:7: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata_atac.var['chr'] = chromosome_start_end_df['chr']


                     chr   start     end
chr1-9776-10668     chr1    9776   10668
chr1-180726-181005  chr1  180726  181005
chr1-181117-181803  chr1  181117  181803
chr1-191133-192055  chr1  191133  192055
chr1-267562-268456  chr1  267562  268456


In [11]:
adata_rna, adata_atac = xc.pp.filter_multiome_data(
    ad_rna=adata_rna,
    ad_atac=adata_atac,
    species='human',
    filter_ratio=0.01
)

RNA data after filtering: View of AnnData object with n_obs × n_vars = 10444 × 12278
    obs: 'cell_type', 'batch', 'Samplename', 'n_genes'
    var: 'feature_types', 'gene_id', 'n_cells'
ATAC data after filtering: View of AnnData object with n_obs × n_vars = 10444 × 53735
    obs: 'cell_type', 'batch', 'Samplename', 'n_genes'
    var: 'feature_types', 'gene_id', 'chr', 'start', 'end', 'n_cells'


In [12]:
train_atac = adata_atac[(adata_atac.obs['Samplename'] == train)]
test_atac = adata_atac[(adata_atac.obs['Samplename'] == test)]

train_rna = adata_rna[(adata_rna.obs['Samplename'] == train)]
test_rna = adata_rna[(adata_rna.obs['Samplename'] == test)]

train_atac.write_h5ad(f'{path}/processed_data/train_atac.h5ad')
test_atac.write_h5ad(f'{path}/processed_data/test_atac.h5ad')
train_rna.write_h5ad(f'{path}/processed_data/train_rna.h5ad')
test_rna.write_h5ad(f'{path}/processed_data/test_rna.h5ad')

/home/miaoyuanyuan/miniconda3/envs/py3.8_tf2.6.0/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/home/miaoyuanyuan/miniconda3/envs/py3.8_tf2.6.0/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
